In [ ]:
!pip install google-generativeai requests youtube-transcript-api

In [37]:
import os
from IPython.display import display, Markdown
from youtube_transcript_api import YouTubeTranscriptApi
import google.generativeai as genai

In [9]:
GOOGLE_API_KEY = os.environ['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)

## Example function: Multiply two numbers

In [15]:
def multiply(a:float, b:float):
    """returns a * b."""
    return a*b

In [16]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro', tools=[multiply])
chat = model.start_chat(enable_automatic_function_calling=True)

In [17]:
response = chat.send_message('I have 57 cats, each owns 44 mittens, how many mittens is that in total?')
response.text

'The total number of mittens is 2508.'

In [18]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'I have 57 cats, each owns 44 mittens, how many mittens is that in total?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'multiply', 'args': {'a': 57.0, 'b': 44.0}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'multiply', 'response': {'result': 2508.0}}}
--------------------------------------------------------------------------------
model -> {'text': 'The total number of mittens is 2508.'}
--------------------------------------------------------------------------------


## Example function: Get currency rates

In [40]:
import requests

def get_exchange_rate(
    currency_from: str = "USD",
    currency_to: str = "EUR",
    currency_date: str = "latest",
):
    """Retrieves the exchange rate between two currencies on a specified date.

    Uses the Frankfurter API (https://api.frankfurter.app/) to obtain exchange rate data.

    Args:
        currency_from: The base currency (3-letter currency code). Defaults to "USD" (US Dollar).
        currency_to: The target currency (3-letter currency code). Defaults to "EUR" (Euro).
        currency_date: The date for which to retrieve the exchange rate. Defaults to "latest"
            for the most recent exchange rate data. Can be specified in YYYY-MM-DD format for
            historical rates.

    Returns:
        dict: A dictionary containing the exchange rate information.
              Example: {"amount": 1.0, "base": "USD", "date": "2023-11-24", "rates": {"EUR": 0.95534}}
    """
    response = requests.get(
        f"https://api.frankfurter.app/{currency_date}",
        params={"from": currency_from, "to": currency_to},
    )
    return response.json()

In [41]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro', tools=[get_exchange_rate])
chat = model.start_chat(enable_automatic_function_calling=True)

In [42]:
response = chat.send_message("What was the exchange rate from US Dollars to Swedish currency as of Feb 1, 2023?")
display(Markdown(response.text))

OK. On February 1, 2023, 1 US Dollar was equivalent to 10.4144 Swedish Krona.

In [43]:
for content in chat.history:
    part = content.parts[0]
    print(content.role, "->", type(part).to_dict(part))
    print('-'*80)

user -> {'text': 'What was the exchange rate from US Dollars to Swedish currency as of Feb 1, 2023?'}
--------------------------------------------------------------------------------
model -> {'function_call': {'name': 'get_exchange_rate', 'args': {'currency_from': 'USD', 'currency_date': '2023-02-01', 'currency_to': 'SEK'}}}
--------------------------------------------------------------------------------
user -> {'function_response': {'name': 'get_exchange_rate', 'response': {'rates': {'SEK': 10.4144}, 'date': '2023-02-01', 'base': 'USD', 'amount': 1.0}}}
--------------------------------------------------------------------------------
model -> {'text': 'OK. On February 1, 2023, 1 US Dollar was equivalent to 10.4144 Swedish Krona.'}
--------------------------------------------------------------------------------


## Example function: Question and answer on YouTube transcripts

In [29]:
def get_youtube_transcript(video_id: str) -> str:
    """Get the subtitles or transcript for a given YouTube video.

    Args:
        video_id (str): The YouTube video ID

    Returns:
        str: Subtitles or transcript of the YouTube video
    """

    response = YouTubeTranscriptApi.get_transcript(video_id)
    return response

In [30]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro', tools=[get_youtube_transcript])
chat = model.start_chat(enable_automatic_function_calling=True)

In [39]:
response = chat.send_message("Based on this highlights video, which players and teams scored the goals during this game? https://www.youtube.com/watch?v=IGQ5TnRv6dg")
display(Markdown(response.text))

**Goalscorers:**

* **Marcus Rashford** (Manchester United) - 14th minute
* **Scott McTominay** (Manchester United) - 29th minute
* **Anthony** (Manchester United) - 73rd minute

**Teams:**

* **Manchester United**
* **Liverpool**

## Example function: Pass parameters to other APIs with lower level `FunctionDeclaration` API

In [45]:
import google.ai.generativelanguage as glm

In [62]:
set_qos_priority = glm.Tool(
    function_declarations=[
      glm.FunctionDeclaration(
        name='set_qos_priority',
        description="Set the QoS priority for a given network service",
        parameters=glm.Schema(
            type=glm.Type.OBJECT,
            properties={
                'app_name':glm.Schema(type=glm.Type.STRING),
                'priority':glm.Schema(type=glm.Type.STRING),
                'ports':glm.Schema(type=glm.Type.NUMBER)
            },
            required=['app_name','priority','ports']
        )
      )
    ])

In [63]:
model = genai.GenerativeModel(model_name='gemini-1.0-pro', tools=[set_qos_priority])
chat = model.start_chat(enable_automatic_function_calling=False)

In [81]:
response = chat.send_message("Prescence detection has determined that that user has left home now, please prioritize QoS for Linux ISO downloads on default BitTorrent ports 6881")
response.candidates[0].content

parts {
  function_call {
    name: "set_qos_priority"
    args {
      fields {
        key: "priority"
        value {
          string_value: "High"
        }
      }
      fields {
        key: "ports"
        value {
          string_value: "6881"
        }
      }
      fields {
        key: "app_name"
        value {
          string_value: "Linux ISO downloads"
        }
      }
    }
  }
}
role: "model"

In [82]:
PREDICTED_FUNCTION_NAME = response.candidates[0].content.parts[0].function_call.name
PREDICTED_FUNCTION_NAME

'set_qos_priority'

In [83]:
PREDICTED_ARGS = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    PREDICTED_ARGS[key] = value
PREDICTED_ARGS

{'ports': '6881', 'app_name': 'Linux ISO downloads', 'priority': 'High'}

In [89]:
# Now you can call ANY function or API in Python with requests or the library of your choice, as in

if PREDICTED_FUNCTION_NAME == "set_qos_priority":
    print(f"Adjusting QoS priority for: {PREDICTED_ARGS['app_name']}")
    print(f"Adjusting QoS priority to: {PREDICTED_ARGS['priority']}")
    print(f"Adjusting QoS priority for ports: {PREDICTED_ARGS['ports']}")
    
    # Example API calling code
    # response = requests.get(
    #     f"https://api.my-router.com/",
    #     params={"ports": PREDICTED_ARGS['ports'], "priority": PREDICTED_ARGS['priority']},
    # )
    # return response.json()

Adjusting QoS priority for: Linux ISO downloads
Adjusting QoS priority to: High
Adjusting QoS priority for ports: 6881
